In [ ]:
from diffusers import DiffusionPipeline
import torch

# pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-cascade", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe = DiffusionPipeline.from_pretrained("jialuliluka/selma-xl", torch_dtype=torch.float16, use_safetensors=True)
pipe.to("cuda")

In [ ]:
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline
import torch
prior = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", variant="bf16", torch_dtype=torch.bfloat16)
decoder = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade", variant="bf16", torch_dtype=torch.float16)

In [ ]:
prompt = "a cat chases a dog"
negative_prompt = ""

prior.enable_model_cpu_offload()
prior_output = prior(
    prompt=prompt,
    height=1024,
    width=1024,
    negative_prompt=negative_prompt,
    guidance_scale=4.0,
    num_images_per_prompt=1,
    num_inference_steps=20
)

decoder.enable_model_cpu_offload()
decoder_output = decoder(
    image_embeddings=prior_output.image_embeddings.to(torch.float16),
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=0.0,
    output_type="pil",
    num_inference_steps=10
).images[0]

display(decoder_output)

In [ ]:
image = pipe('a cat chases a dog').images[0]
display(image)